In [1]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install -U scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 1.7 MB/s eta 0:00:07
   --- ------------------------------------ 1.0/11.1 MB 1.7 MB/s eta 0:00:06
   ---- ----------------------------------- 1.3/11.1 MB 1.7 MB/s eta 0:00:06
   ----- ---------------------------------- 1.6/11.1 MB 1.7 MB/s eta 0:00:06
   ------- -------------------------------- 2.1/11.1 MB 1.7 MB/s eta 0:00:06
   -------- ------------------------------- 2.4/11.1 MB 1.7 MB/s eta 0:00:06
   --------- ------------------------------ 2.6/11.1 MB 1.7 MB/s eta 0:00:06
   ----------- ---------------------------- 3.1/11.1 MB 1.7 MB/s eta 0:00:05
   ------------ --------------------------- 3.4/11.1 MB 1.7 MB/s eta 0:00:05
   ------------- -------------------------- 3.7/11.1 MB 1.7 MB/s eta 0:00:05
   --------------- ------------------------ 4.2/11.1 MB 1.7 MB/s eta 0:00:05
   ----------


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Carregar os dados
#dados = pd.read_csv('seu_arquivo.csv', parse_dates=['data'])

In [6]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
credit_approval = fetch_ucirepo(id=27) 
  
# data (as pandas dataframes) 
X = credit_approval.data.features 
y = credit_approval.data.targets 
  
# metadata 
print(credit_approval.metadata) 
  
# variable information 
print(credit_approval.variables) 


{'uci_id': 27, 'name': 'Credit Approval', 'repository_url': 'https://archive.ics.uci.edu/dataset/27/credit+approval', 'data_url': 'https://archive.ics.uci.edu/static/public/27/data.csv', 'abstract': 'This data concerns credit card applications; good mix of attributes', 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 690, 'num_features': 15, 'feature_types': ['Categorical', 'Integer', 'Real'], 'demographics': [], 'target_col': ['A16'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1987, 'last_updated': 'Wed Aug 23 2023', 'dataset_doi': '10.24432/C5FS30', 'creators': ['J. R. Quinlan'], 'intro_paper': None, 'additional_info': {'summary': 'This file concerns credit card applications.  All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data.\r\n  \r\nThis dataset is interesting because there is a good mix of attributes --

In [10]:
df = pd.concat([X, y], axis=1)
print(df.head())

   A15    A14 A13 A12  A11 A10 A9    A8 A7 A6 A5 A4     A3     A2 A1 A16
0    0  202.0   g   f    1   t  t  1.25  v  w  g  u  0.000  30.83  b   +
1  560   43.0   g   f    6   t  t  3.04  h  q  g  u  4.460  58.67  a   +
2  824  280.0   g   f    0   f  t  1.50  h  q  g  u  0.500  24.50  a   +
3    3  100.0   g   t    5   t  t  3.75  v  w  g  u  1.540  27.83  b   +
4    0  120.0   s   f    0   f  t  1.71  v  w  g  u  5.625  20.17  b   +


In [12]:
# Verificar o nome atual da coluna target
print("Colunas do DataFrame:", df.columns.tolist())

# Se o target estiver com nome genérico como 'Class' ou 'Target', podemos renomear
df = df.rename(columns={df.columns[-1]: 'Credit_Approval'})  # Assumindo que o target é a última coluna

Colunas do DataFrame: ['A15', 'A14', 'A13', 'A12', 'A11', 'A10', 'A9', 'A8', 'A7', 'A6', 'A5', 'A4', 'A3', 'A2', 'A1', 'A16']


In [13]:
# Substituir '?' por NaN
df.replace('?', pd.NA, inplace=True)

# Codificar variáveis categóricas
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Credit_Approval'] = le.fit_transform(df['Credit_Approval'])

# One-hot encoding para outras variáveis categóricas
df = pd.get_dummies(df, drop_first=True)

In [14]:
print("\nInformações do DataFrame:")
print(df.info())

print("\nEstatísticas descritivas:")
print(df.describe(include='all'))

print("\nValores ausentes por coluna:")
print(df.isnull().sum())


Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 38 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   A15              690 non-null    int64  
 1   A14              677 non-null    float64
 2   A11              690 non-null    int64  
 3   A8               690 non-null    float64
 4   A3               690 non-null    float64
 5   A2               678 non-null    float64
 6   Credit_Approval  690 non-null    int64  
 7   A13_p            690 non-null    bool   
 8   A13_s            690 non-null    bool   
 9   A12_t            690 non-null    bool   
 10  A10_t            690 non-null    bool   
 11  A9_t             690 non-null    bool   
 12  A7_dd            690 non-null    bool   
 13  A7_ff            690 non-null    bool   
 14  A7_h             690 non-null    bool   
 15  A7_j             690 non-null    bool   
 16  A7_n             690 non-null    bo

In [15]:
# Para colunas numéricas, preencher com mediana
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Para colunas categóricas, preencher com moda
cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Verificar novamente valores ausentes
print("\nValores ausentes após tratamento:")
print(df.isnull().sum())


Valores ausentes após tratamento:
A15                0
A14                0
A11                0
A8                 0
A3                 0
A2                 0
Credit_Approval    0
A13_p              0
A13_s              0
A12_t              0
A10_t              0
A9_t               0
A7_dd              0
A7_ff              0
A7_h               0
A7_j               0
A7_n               0
A7_o               0
A7_v               0
A7_z               0
A6_c               0
A6_cc              0
A6_d               0
A6_e               0
A6_ff              0
A6_i               0
A6_j               0
A6_k               0
A6_m               0
A6_q               0
A6_r               0
A6_w               0
A6_x               0
A5_gg              0
A5_p               0
A4_u               0
A4_y               0
A1_b               0
dtype: int64


In [ ]:
def criar_target(df, mes_referencia):
    """
    Cria a variável target para um mês de referência
    
    Parâmetros:
    df: DataFrame com os dados
    mes_referencia: mês de referência no formato 'YYYY-MM'
    
    Retorna:
    Series com a target (1 se contratou crédito nos 3 meses seguintes, 0 caso contrário)
    """
    inicio = pd.to_datetime(mes_referencia)
    fim = inicio + pd.DateOffset(months=3)
    
    # Identificar clientes que contrataram no período
    contratantes = df[(df['data'] > inicio) & 
                     (df['data'] <= fim) & 
                     (df['contratou_credito'] == 1)]['id_cliente'].unique()
    
    # Criar target
    target = df[df['data'] == inicio]['id_cliente'].isin(contratantes).astype(int)
    
    return target

# Exemplo para abril
target_abril = criar_target(dados, '2023-04-01')

In [ ]:
def criar_features_autorregressivas(df, id_cliente, data_referencia, meses_hist=[1, 2, 3]):
    """
    Cria features autorregressivas para um cliente em uma data específica
    
    Parâmetros:
    df: DataFrame com os dados
    id_cliente: ID do cliente
    data_referencia: data de referência
    meses_hist: lista de meses históricos para calcular as estatísticas
    
    Retorna:
    DataFrame com as features para o cliente na data de referência
    """
    data_ref = pd.to_datetime(data_referencia)
    features = {}
    
    for meses in meses_hist:
        # Filtrar dados históricos
        historico = df[(df['id_cliente'] == id_cliente) & 
                       (df['data'] <= data_ref) & 
                       (df['data'] > data_ref - pd.DateOffset(months=meses))]
        
        # Calcular estatísticas
        if not historico.empty:
            features[f'limite_utilizado_media_{meses}m'] = historico['limite_utilizado'].mean()
            features[f'limite_utilizado_min_{meses}m'] = historico['limite_utilizado'].min()
            features[f'limite_utilizado_max_{meses}m'] = historico['limite_utilizado'].max()
            features[f'qtd_transacoes_{meses}m'] = historico.shape[0]
        else:
            # Preencher com zeros se não houver histórico
            features[f'limite_utilizado_media_{meses}m'] = 0
            features[f'limite_utilizado_min_{meses}m'] = 0
            features[f'limite_utilizado_max_{meses}m'] = 0
            features[f'qtd_transacoes_{meses}m'] = 0
    
    return pd.DataFrame(features, index=[0])

# Exemplo de aplicação para todos os clientes em um mês específico
def criar_dataset_mes(df, mes_referencia):
    data_ref = pd.to_datetime(mes_referencia)
    clientes_mes = df[df['data'] == data_ref]['id_cliente'].unique()
    
    features_list = []
    for cliente in clientes_mes:
        features = criar_features_autorregressivas(df, cliente, data_ref)
        features['id_cliente'] = cliente
        features['data'] = data_ref
        features_list.append(features)
    
    return pd.concat(features_list, ignore_index=True)

In [ ]:
# Criar datasets para cada mês
meses_treino = ['2023-04-01', '2023-05-01', '2023-06-01']
meses_teste = ['2023-07-01', '2023-08-01']

# Criar features e targets para treino
X_treino_list = []
y_treino_list = []

for mes in meses_treino:
    # Criar features
    features_mes = criar_dataset_mes(dados, mes)
    X_treino_list.append(features_mes)
    
    # Criar target
    target_mes = criar_target(dados, mes)
    y_treino_list.append(target_mes)

X_treino = pd.concat(X_treino_list)
y_treino = pd.concat(y_treino_list)

# Criar features e targets para teste
X_teste_list = []
y_teste_list = []

for mes in meses_teste:
    features_mes = criar_dataset_mes(dados, mes)
    X_teste_list.append(features_mes)
    
    target_mes = criar_target(dados, mes)
    y_teste_list.append(target_mes)

X_teste = pd.concat(X_teste_list)
y_teste = pd.concat(y_teste_list)

# Remover colunas não utilizadas no modelo
cols_modelo = [col for col in X_treino.columns if col not in ['id_cliente', 'data']]
X_treino = X_treino[cols_modelo]
X_teste = X_teste[cols_modelo]

In [ ]:
# Juntar X e y para balanceamento
df_treino = pd.concat([X_treino, y_treino], axis=1)

# Separar classes
classe_maioria = df_treino[df_treino['target'] == 0]
classe_minoria = df_treino[df_treino['target'] == 1]

# Undersampling da classe majoritária
classe_maioria_downsampled = resample(classe_maioria,
                                     replace=False,
                                     n_samples=len(classe_minoria),
                                     random_state=42)

# Combinar as classes balanceadas
df_treino_balanced = pd.concat([classe_maioria_downsampled, classe_minoria])

# Separar novamente X e y
X_treino_balanced = df_treino_balanced.drop('target', axis=1)
y_treino_balanced = df_treino_balanced['target']

In [ ]:
# Criar pipeline com normalização e modelo
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(class_weight='balanced', 
                                random_state=42,
                                max_iter=1000))
])

# Treinar o modelo
pipeline.fit(X_treino_balanced, y_treino_balanced)

# Previsões no conjunto de teste
y_pred = pipeline.predict(X_teste)
y_pred_proba = pipeline.predict_proba(X_teste)[:, 1]

# Avaliação
print(classification_report(y_teste, y_pred))
print(f"AUC-ROC: {roc_auc_score(y_teste, y_pred_proba)}")

In [ ]:
# Extrair coeficientes do modelo
coeficientes = pipeline.named_steps['model'].coef_[0]
features_importance = pd.DataFrame({
    'Feature': cols_modelo,
    'Importance': coeficientes
}).sort_values('Importance', ascending=False)

print(features_importance)

In [ ]:
def prever_propensao(df, mes_referencia, modelo):
    """
    Prever propensão para contratação de crédito para um mês específico
    
    Parâmetros:
    df: DataFrame com os dados
    mes_referencia: mês de referência no formato 'YYYY-MM'
    modelo: modelo treinado
    
    Retorna:
    DataFrame com IDs dos clientes e suas probabilidades de propensão
    """
    # Criar features
    features = criar_dataset_mes(df, mes_referencia)
    
    # Fazer previsões
    X = features[cols_modelo]
    features['probabilidade'] = modelo.predict_proba(X)[:, 1]
    
    return features[['id_cliente', 'data', 'probabilidade']].sort_values('probabilidade', ascending=False)

# Exemplo de uso
propensao_setembro = prever_propensao(dados, '2023-09-01', pipeline)